In [2]:
import os, sys
from pathlib import Path

def find_src_folder():
    current = Path(os.getcwd()).resolve()
    for p in [current] + list(current.parents):
        src = p / "src"
        if src.exists():
            return src
    raise RuntimeError("src 폴더를 찾을 수 없습니다.")

src_path = find_src_folder()
sys.path.append(str(src_path))

## BGM3_TEST
- 레이턴시 0.1~0.2초

In [ ]:
from pypdf import PdfReader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from core.common import get_src_path
pdf_path = get_src_path() / "tests"/"fairy"/"2501.03468v1.pdf"

def load_pdf(path):
    reader = PdfReader(path)
    texts = []
    for page in reader.pages:
        text = page.extract_text()
        if text:
            texts.append(text)
    return "\n".join(texts)

raw_text = load_pdf(pdf_path)
print("📄 PDF 텍스트 길이:", len(raw_text))


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,       # bge-m3는 길이 강함. 800~1500 추천
    chunk_overlap=200,
)

chunks = text_splitter.split_text(raw_text)

from FlagEmbedding import BGEM3FlagModel
model = BGEM3FlagModel("BAAI/bge-m3", use_fp16=True)

import chromadb
client = chromadb.PersistentClient(path="./chroma_bge_m3_demo")
collection = client.get_or_create_collection(name="bge-m3-test")

ids = [f"chunk-{i}" for i in range(len(chunks))]
metas = [{"source": "2501.03468v1", "chunk_id": i} for i in range(len(chunks))]
emb = model.encode(
    chunks,
    batch_size=8,
    max_length=8192,
    return_dense=True,
    return_sparse=False,
    return_colbert_vecs=False,
)

dense_vecs = emb["dense_vecs"].tolist()

collection.add(
    ids=ids,
    documents=chunks,
    embeddings=dense_vecs,
    metadatas=metas,
)
print(f"🔹 청크 개수: {len(chunks)}")


In [17]:
query_text = "이 논문에서 multi-turn agent 구조는 어떻게 설명되고 있는가?"

query_emb = model.encode(
    [query_text],
    return_dense=True,
    return_sparse=False,
    return_colbert_vecs=False,
)["dense_vecs"].tolist()

# 8) Chroma에서 벡터 검색
results = collection.query(
    query_embeddings=query_emb,
    n_results=1,
)
print(results)

{'ids': [['chunk-29']], 'embeddings': None, 'documents': [['Subset R@5\nBy Turn Turn 1 (102) 0.89\n> Turn 1 (675) 0.47\nBy Standalone Standalone (555) 0.48\n(> Turn 1) Non-Standalone (120) 0.42\nBy Domain CLAPNQ (208) 0.56\nFiQA (180) 0.50\nGovt (201) 0.56\nCloud (188) 0.47\nTable 4: Elser retrieval results with query rewrite on\nsubsets of the data to highlight multi-turn properties.\nNumbers in parentheses denote size of each subset.\n6 Generation\nWe next present the generator experiments. We\nstart with the experimental setup, followed by the\nresults, using automated metrics including LLM\njudges. Section 7 will complement this with a hu-\nman evaluation on a subset of MTRAG. Given a\ntask, we send to the model the following informa-\ntion: the question, preceding turns, N passages, and\ninstructions. We choose N=5 passages because it\nachieves considerable improvement compared to\ntop 3, while remaining a manageable amount of\npassages (Section 5). For more generation format\ndet

# OPEN AI TEST
- 레이턴시 0.3초

In [12]:


from langchain_openai import OpenAIEmbeddings
import chromadb

openai_embedder = OpenAIEmbeddings(
    model="text-embedding-3-large"   
)

client_openai = chromadb.PersistentClient(path="./chroma_openai_demo")


# collection_openai = client_openai.get_or_create_collection(
#     name="openai-embed-test"
# )
collection_openai = client_openai.get_or_create_collection(
    name="openai-embed-test-large",   # 변경!
)
openai_vectors = openai_embedder.embed_documents(chunks)
ids_openai = [f"openai-chunk-{i}" for i in range(len(chunks))]
metas_openai = [{"source": "2501.03468v1", "chunk_id": i} for i in range(len(chunks))]

collection_openai.add(
    ids=ids_openai,
    documents=chunks,
    embeddings=openai_vectors,
    metadatas=metas_openai,
)

print("✅ OpenAI Embedding 기반 Chroma 저장 완료!")

✅ OpenAI Embedding 기반 Chroma 저장 완료!


In [13]:
query_text = "이 논문에서 multi-turn agent 구조는 어떻게 설명되는가?"

query_emb_openai = openai_embedder.embed_query(query_text)

results_openai = collection_openai.query(
    query_embeddings=[query_emb_openai],
    n_results=2,
)

print("\n=== 🔍 OpenAI Embedding 검색 결과 ===")
for doc, meta, dist in zip(
    results_openai["documents"][0],
    results_openai["metadatas"][0],
    results_openai["distances"][0],
):
    print("\n--------------------------------")
    print("📌 Chunk:", meta["chunk_id"])
    print("📄 내용:", doc[:200], "...")
    print("📉 거리:", dist)


=== 🔍 OpenAI Embedding 검색 결과 ===

--------------------------------
📌 Chunk: 26
📄 내용: We experimented with several strategies to query
the retriever for relevant passages, including
sending the full conversation up to the current
user turn, all the user turns without the responses,
sub ...
📉 거리: 1.0796127319335938

--------------------------------
📌 Chunk: 21
📄 내용: generator outputs as needed. Using the application,
annotators created multi-turn conversations by per-
forming the following actions at every turn: write
a question, adjust the set of retrieved passa ...
📉 거리: 1.1401104927062988
